In [61]:
import requests
import pandas as pd
from IPython.display import display, HTML

In [2]:
def fetch_graphql_data(endpoint, query, headers=None):
    response = requests.get(endpoint, json={'query': query})
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}")
        return None

In [3]:
def format_dataframe(df):
    # Custom styles
    styles = {
        # Left-align index
        'selector': 'th',
        'props': [('text-align', 'left')]
    }
    
    # Format 'Total Votes' and apply styles
    return (df.style
            .format({'Total Votes': "{:,.0f}"})
            .set_table_styles([styles]))

In [45]:
def extract_choices(choice_data):
    return {str(key): " ".join(value.split(' - ')[1:]).strip() for key, value in enumerate(choice_data['data']['proposal']['choices'], start=1)}


In [57]:
def results(wg='eco'):
    choice_query = f"""
    query {{
        proposal(id: "{proposal_ids[wg]}") {{
            votes
            choices
            scores
        }}
    }}
    """

    results_query = f"""
    query {{
        votes(first: 1000, where: {{ proposal: "{proposal_ids[wg]}" }}) {{
            voter
            vp
            choice
        }}
    }}
    """
    
    #get data from snapshot
    
    choice_data = fetch_graphql_data(endpoint, choice_query)
    results_data = fetch_graphql_data(endpoint, results_query)
    
    #process data
    choices = extract_choices(choice_data)
    
    results_data = fetch_graphql_data(endpoint, results_query)
    df = pd.DataFrame(results_data['data']['votes'])
    df = pd.json_normalize(df.to_dict('records'))
    voter_choices = df.fillna(0).set_index('voter').iloc[:, 1:].rename(columns={f'choice.{i}': str(i) for i in range(1, len(choices) + 1)})
    voter_choices['total'] = voter_choices.sum(axis=1) 
    
    for col in voter_choices.iloc[:,:-1].columns:
        voter_choices[col] = (voter_choices[col] / voter_choices['total'])*100
    voter_choices = voter_choices.iloc[:,:-1]
    column_sum = voter_choices.sum()


    non_zero_count = voter_choices.apply(lambda x: (x != 0).sum())


    result = pd.DataFrame({
    'Total Votes': column_sum,
    '# Voters': non_zero_count
    })
    result = result.rename(index=choices)
    
    return format_dataframe(result.sort_values('Total Votes',ascending=False))

In [63]:
def display_with_title(df, title):
    title_html = f"<h3>{title}</h3>"
    df_html = df.to_html()
    display(HTML(title_html + df_html))

In [4]:
endpoint = "https://hub.snapshot.org/graphql"

In [16]:
proposal_ids = {'eco':"0xfd2b613847557ba419622d480939cb12d880df6f648f6e4489095a71c3002543",
               'pg':'0x1f9519840ebb3974ad5ccb45edf9865e1a5710a2f27ba13ca147251311fdfa26'}

In [64]:
display_with_title(results('pg'),"Small Grants - Public Goods")

,Total Votes,# Voters
Giveth Building the Future of Giving,"1,511",43
Ethereum News by @ethdaily,"1,285",31
dm3 the web3 messaging interoperability protocol,"1,282",36
rotki,"1,166",35
"Pairwise: Simplifying Choices, Amplifying Voices",645,20
EIP-7212 Research,509,24
Arabic Web3 Education,358,15
MetaGame,349,12
BrightID,340,16
Unitap,340,9


In [66]:
display_with_title(results('eco'),"Small Grants - Ecosystem")

,Total Votes,# Voters
ENS Wayback Machine,"1,639",25
1W3 decentralised websites for your ENS,"1,483",37
"Namespace Subname Search, Registration & Issuing","1,303",35
Smart Talk Tech + Talky + ENS = Decentralizing Conversations with AI Intelligence,"1,170",19
NameSys,849,27
L2 ENS Subnames + ERC-4337,576,20
Web3Domains.com | News & Visibility,554,20
HEAVENS TOOLS L.L.C.,488,14
Avatar Sync,487,16
NameGuard,456,20
